In [1]:
import pandas as pd
from unidecode import unidecode

In [2]:
def carregar_e_mesclar_arquivos(url_imp_mun_2023: str, url_ncm_sh: str, 
                                chave_principal: str, chave_referencia: str, 
                                sep_principal: str = ',', sep_referencia: str = ';', 
                                encoding: str = 'latin1'):

    df_imp_mun_2023 = pd.read_csv(url_imp_mun_2023, sep=sep_principal, encoding=encoding)
    df_ncm_sh = pd.read_csv(url_ncm_sh, sep=sep_referencia, encoding=encoding)

    df_imp_mun_2023[chave_principal] = df_imp_mun_2023[chave_principal].astype(str)
    df_ncm_sh[chave_referencia] = df_ncm_sh[chave_referencia].astype(str)

    df_resultado = df_imp_mun_2023.merge(df_ncm_sh, left_on=chave_principal, right_on=chave_referencia, how='left')

    return df_resultado

df_1 = carregar_e_mesclar_arquivos(
    url_imp_mun_2023='/home/edcarlos/Data-Analytics-Comercio-Exterior/datalake/landing/importacao_mun_2023.csv',
    url_ncm_sh='/home/edcarlos/Data-Analytics-Comercio-Exterior/datalake/raw_data/NCM_SH.csv',
    chave_principal='cd_sh4',
    chave_referencia='CO_SH4'
)


In [3]:
df_1 = df_1.drop(columns=['cd_sh4', 'CO_SH6', 'NO_SH6_POR', 'NO_SH6_ESP',
       'NO_SH6_ING', 'CO_SH4', 'NO_SH4_ESP', 'NO_SH4_ING',
       'CO_SH2', 'NO_SH2_POR', 'NO_SH2_ESP', 'NO_SH2_ING', 'CO_NCM_SECROM',
       'NO_SEC_POR', 'NO_SEC_ESP', 'NO_SEC_ING'])

In [4]:
def carregar_e_mesclar_2(df_1: pd.DataFrame, caminho_referencia: str, 
                       chave_base: str, chave_referencia: str, 
                       sep_referencia: str = ';', encoding: str = 'latin1') -> pd.DataFrame:

    df_referencia = pd.read_csv(caminho_referencia, sep=sep_referencia, encoding=encoding)

    df_1[chave_base] = df_1[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    df_resto = df_1.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resto

arquivo_pais = '/home/edcarlos/Data-Analytics-Comercio-Exterior/datalake/raw_data/PAIS.csv'

df_1 = carregar_e_mesclar_2(df_1, arquivo_pais, chave_base='cd_pais', chave_referencia='CO_PAIS')


In [5]:
df_2 = df_1.drop(columns=['cd_pais', 'CO_PAIS', 'CO_PAIS_ISON3',
       'CO_PAIS_ISOA3', 'NO_PAIS_ING', 'NO_PAIS_ESP'])

In [6]:
caminho_uf_mun = '/home/edcarlos/Data-Analytics-Comercio-Exterior/datalake/raw_data/UF_MUN.csv' 
df_uf_mun = pd.read_csv(caminho_uf_mun, sep=';',  encoding='latin1') 

In [7]:
def mesclar_dataframes(df_base: pd.DataFrame, df_referencia: pd.DataFrame, 
                       chave_base: str, chave_referencia: str) -> pd.DataFrame:
  
    df_base[chave_base] = df_base[chave_base].astype(str)
    df_referencia[chave_referencia] = df_referencia[chave_referencia].astype(str)

    df_resultante = df_base.merge(df_referencia, left_on=chave_base, right_on=chave_referencia, how='left')
    
    return df_resultante

df_2 = mesclar_dataframes(df_base=df_2, df_referencia=df_uf_mun, chave_base='cd_mun', chave_referencia='CO_MUN_GEO')

In [8]:
df_2 = df_2.drop(columns=['cd_mun', 'CO_MUN_GEO', 'NO_MUN', 'SG_UF'])

In [9]:
df_3 = df_2.rename(columns={  
    'sg_ufmun': 'sg_uf',
    'data_importacao_mun': 'data_importacao',
    'NO_SH4_POR': 'cd_ncm', 
    'NO_PAIS': 'cd_pais', 
    'NO_MUN_MIN': 'cd_mun'
    })
df_3.columns = df_3.columns.str.strip().str.lower().str.replace(' ', '_')

In [10]:
df_3['cd_pais'] = df_3['cd_pais'].str.lower()
df_3['cd_mun'] = df_3['cd_mun'].str.lower()

In [11]:
for col in df_3.select_dtypes(include='object').columns:
    df_3[col] = df_3[col].apply(lambda x: unidecode(x) if isinstance(x, str) else x)

In [12]:
caminho = "/home/edcarlos/Data-Analytics-Comercio-Exterior/datalake/silver/importacao_mun_2023_m.csv"
df_3.to_csv(caminho, index=False)